In [1]:

import os


import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

spark = SparkSession.builder.master("local[*]") \
    .appName("testing") \
    .config("spark.driver.extraClassPath", "C:\\my_sql_jar\\mysql-connector-java-8.0.26.jar") \
    .getOrCreate()
print(spark)

In [2]:
emp_data = [
(1,'manish',26,20000,'india','IT'),
(2,'rahul',None,40000,'germany','engineering'),
(3,'pawan',12,60000,'india','sales'),
(4,'roshini',44,None,'uk','engineering'),
(5,'raushan',35,70000,'india','sales'),
(6,None,29,200000,'uk','IT'),
(7,'adam',37,65000,'us','IT'),
(8,'chris',16,40000,'us','sales'),
(None,None,None,None,None,None),
(7,'adam',37,65000,'us','IT')
]

emp_schema = ['id','name', 'age', 'salary', 'country', 'dept']

emp_df = spark.createDataFrame(data = emp_data, schema = emp_schema)
emp_df.show()

+----+-------+----+------+-------+-----------+
|  id|   name| age|salary|country|       dept|
+----+-------+----+------+-------+-----------+
|   1| manish|  26| 20000|  india|         IT|
|   2|  rahul|null| 40000|germany|engineering|
|   3|  pawan|  12| 60000|  india|      sales|
|   4|roshini|  44|  null|     uk|engineering|
|   5|raushan|  35| 70000|  india|      sales|
|   6|   null|  29|200000|     uk|         IT|
|   7|   adam|  37| 65000|     us|         IT|
|   8|  chris|  16| 40000|     us|      sales|
|null|   null|null|  null|   null|       null|
|   7|   adam|  37| 65000|     us|         IT|
+----+-------+----+------+-------+-----------+



In [5]:
# adding extra column named "adult", in this column checking if each record is adult or not using WHEN/OTHERWISE condition

emp_df.withColumn("adult", when(col("age")<18, "No")
                           .when(col("age")>18, "Yes")
                           .otherwise("No value")).show()

+----+-------+----+------+-------+-----------+--------+
|  id|   name| age|salary|country|       dept|   adult|
+----+-------+----+------+-------+-----------+--------+
|   1| manish|  26| 20000|  india|         IT|     Yes|
|   2|  rahul|null| 40000|germany|engineering|No value|
|   3|  pawan|  12| 60000|  india|      sales|      No|
|   4|roshini|  44|  null|     uk|engineering|     Yes|
|   5|raushan|  35| 70000|  india|      sales|     Yes|
|   6|   null|  29|200000|     uk|         IT|     Yes|
|   7|   adam|  37| 65000|     us|         IT|     Yes|
|   8|  chris|  16| 40000|     us|      sales|      No|
|null|   null|null|  null|   null|       null|No value|
|   7|   adam|  37| 65000|     us|         IT|     Yes|
+----+-------+----+------+-------+-----------+--------+



In [7]:
# fixing all the null values first by giving default value as 20 if null then adding adult column 

emp_df.withColumn("age", when(col("age").isNull(), lit(20))
                         .otherwise(col("age")))\
      .withColumn("adult", when(col("age")<18, "No")
                           .when(col("age")>18, "Yes")
                           .otherwise("No value")).show()

+----+-------+---+------+-------+-----------+-----+
|  id|   name|age|salary|country|       dept|adult|
+----+-------+---+------+-------+-----------+-----+
|   1| manish| 26| 20000|  india|         IT|  Yes|
|   2|  rahul| 20| 40000|germany|engineering|  Yes|
|   3|  pawan| 12| 60000|  india|      sales|   No|
|   4|roshini| 44|  null|     uk|engineering|  Yes|
|   5|raushan| 35| 70000|  india|      sales|  Yes|
|   6|   null| 29|200000|     uk|         IT|  Yes|
|   7|   adam| 37| 65000|     us|         IT|  Yes|
|   8|  chris| 16| 40000|     us|      sales|   No|
|null|   null| 20|  null|   null|       null|  Yes|
|   7|   adam| 37| 65000|     us|         IT|  Yes|
+----+-------+---+------+-------+-----------+-----+



In [8]:
emp_df.withColumn("age_wise", when((col("age")>0) & (col("age")<18), "Minor")
                              .when((col("age")>=18) & (col("age")<30), "Mid")
                              .otherwise("Senior"))\
                              .show()

+----+-------+----+------+-------+-----------+--------+
|  id|   name| age|salary|country|       dept|age_wise|
+----+-------+----+------+-------+-----------+--------+
|   1| manish|  26| 20000|  india|         IT|     Mid|
|   2|  rahul|null| 40000|germany|engineering|  Senior|
|   3|  pawan|  12| 60000|  india|      sales|   Minor|
|   4|roshini|  44|  null|     uk|engineering|  Senior|
|   5|raushan|  35| 70000|  india|      sales|  Senior|
|   6|   null|  29|200000|     uk|         IT|     Mid|
|   7|   adam|  37| 65000|     us|         IT|  Senior|
|   8|  chris|  16| 40000|     us|      sales|   Minor|
|null|   null|null|  null|   null|       null|  Senior|
|   7|   adam|  37| 65000|     us|         IT|  Senior|
+----+-------+----+------+-------+-----------+--------+

